In [1]:
import sys
from pathlib import Path
import IMP
import IMP.atom

import IMP
import IMP.core
import IMP.algebra


In [2]:
m = IMP.Model()
pdb_file = Path(Path.home(), "Documents/xray/data/pdbs/7mhf/7mhf.pdb")

h = IMP.atom.read_pdb(str(pdb_file), m, IMP.atom.NonAlternativePDBSelector())

params_file = Path("/Users/matthew/opt/anaconda3/envs/imp_221_cctbx/share/IMP/atom/par.lib")
topology_file = Path("/Users/matthew/opt/anaconda3/envs/imp_221_cctbx/share/IMP/atom/top.lib")

# ff = IMP.atom.CHARMMParameters(str(topology_file), str(params_file), True)
# topology = ff.create_topology(h)

ff = IMP.atom.get_all_atom_CHARMM_parameters()
topology = ff.create_topology(h)

topology.apply_default_patches()
topology.add_atom_types(h)

begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET: O  " in residue #659 "HOH"
  WARNING  Could not determine CHARMM atom type for atom "H1" in residue #1 "SER"
  WARNING  Could not determine CHARMM atom type for atom "HE2" in residue #41 "HIS"
  WARNING  Could not determine CHARMM atom type for atom "HET: S  " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: O  " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: C1 " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: C2 " in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: H11" in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: H12" in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: H13" in residue #401 "DMS"
  WARNING  Could not determine CHARMM atom type for atom "HET: H21" in residue #401 "DMS"
  WARNIN

In [3]:
rs = list()
# topology.add_missing_atoms(h)
bonds = topology.add_bonds(h)
angles = ff.create_angles(bonds)
dihedrals = ff.create_dihedrals(bonds)
impropers = topology.add_impropers(h)
# charges = topology.add_charges(h)

# Add a restraint on the bond lengths.
cont = IMP.container.ListSingletonContainer(m, bonds, "bnd")
bss = IMP.atom.BondSingletonScore(IMP.core.Harmonic(0, 1))
r = IMP.container.SingletonsRestraint(bss, cont, "bnd")
rs.append(r)

# Add a restraint on the bond angles.
cont = IMP.container.ListSingletonContainer(m, angles, "ang")
bss = IMP.atom.AngleSingletonScore(IMP.core.Harmonic(0, 1))
r = IMP.container.SingletonsRestraint(bss, cont, "ang")
rs.append(r)

# Add a restraint on the dihedral angles.
cont = IMP.container.ListSingletonContainer(m, dihedrals, "dih")
bss = IMP.atom.DihedralSingletonScore()
r = IMP.container.SingletonsRestraint(bss, cont, "dih")
rs.append(r)

# Add a restraint on the improper dihedrals (out of plane bending).
cont = IMP.container.ListSingletonContainer(m, impropers, "imp")
bss = IMP.atom.ImproperSingletonScore(IMP.core.Harmonic(0, 1))
rs.append(IMP.container.SingletonsRestraint(bss, cont, "imp"))

# Add a restraint on the non-bonded atoms (Lennard-Jones potential).
ff.add_radii(h)
ff.add_well_depths(h)
atoms = IMP.atom.get_by_type(h, IMP.atom.ATOM_TYPE)
cont = IMP.container.ListSingletonContainer(m, atoms)
nbl = IMP.container.ClosePairContainer(cont, 5, 1)

pair_filter = IMP.atom.StereochemistryPairFilter()
pair_filter.set_bonds(bonds)
pair_filter.set_angles(angles)
pair_filter.set_dihedrals(dihedrals)
nbl.add_pair_filter(pair_filter)
sf = IMP.atom.ForceSwitch(6.0, 7.0)
ljps = IMP.atom.LennardJonesPairScore(sf)
rs.append(IMP.container.PairsRestraint(ljps, nbl, "nbd"))


In [4]:
rs[-1].evaluate(False)

23220885.915432308

In [5]:
# for i in range(len(nbl.get_indexes())):
#     score = ljps.evaluate_index(m, nbl.get_indexes()[i], None)

#     if score > 100000:
#         print(i, IMP.atom.Atom(m, nbl.get_indexes()[i][0]).get_name(), IMP.atom.Atom(m, nbl.get_indexes()[i][1]).get_name(), score)


0 Atom N of residue 1 Atom H1 of residue 1 2293283.909957532


KeyboardInterrupt: 

In [6]:
# for i in range(len(nbl.get_indexes())):
#     score = ljps.evaluate_index(m, nbl.get_indexes()[i], None)

#     if score > 100000:
#         print(i, IMP.atom.Atom(m, nbl.get_indexes()[i][0]).get_name(), IMP.atom.Atom(m, nbl.get_indexes()[i][1]).get_name(), score)


In [7]:
cont.get_contents()

array([   0,    5,    6, ..., 5664, 5666, 5668], dtype=int32)

In [8]:
nbl.get_slack()

1.0

In [9]:
nbl.get_indexes()[18948]

array([ 103, 5019], dtype=int32)

In [10]:
pids = m.get_particle_indexes()

In [11]:
nbl.get_all_possible_indexes()

array([   0,    5,    6, ..., 5664, 5666, 5668], dtype=int32)

In [12]:
sf = IMP.core.RestraintsScoringFunction(rs)
cg = IMP.core.ConjugateGradients(m)
cg.set_scoring_function(sf)

cg.optimize(100)

1314.885195616768

In [16]:
nbl.get_number_of_full_rebuilds()

1

In [14]:
m.get_dependencies_updated()

2

In [6]:
IMP.atom.write_pdb(h, "output.pdb")